In [20]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="jNBAnadLe4NRzxqNNSc0")
project = rf.workspace("jackyzjh").project("soccer-penalty")
version = project.version(2)
dataset = version.download("yolov11")


loading Roboflow workspace...
loading Roboflow project...


In [21]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 42.2/112.6 GB disk)


In [22]:
!yolo train model=yolo11n.pt data=/content/Soccer-Penalty-2/data.yaml epochs=50 imgsz=640

Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Soccer-Penalty-2/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pr

In [ ]:
from ultralytics import YOLO

# Load the best model weights
model = YOLO('commitment/detect/train/weights/best.pt')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
for folder in ["1xx", "x1x", "xx1"]:
    results = model.predict(
        source=f"/commitment/baseline/laliga penalty kicks edited/{folder}", #replace this with your dataset folder
        imgsz=640,
        save=True,
        save_txt=True,
        project="runs/predict",
        name=f"labels_{folder}"
    )


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/9 (frame 1/28) /content/drive/MyDrive/APS360/laliga penalty kicks edited/1xx/alvarez.mp4: 384x640 1 ball, 4 goalframes, 1 goalie, 1 kicker, 16.5ms
video 1/9 (frame 2/28) /content/drive/MyDrive/APS360/laliga penalty kicks edited/1xx/alvarez.mp4: 384x640 1 ball, 4 goalframes, 1 goalie, 1 kicker, 14.3ms
video 1/9 (frame 3/28) /content/drive/MyDrive/APS360/laliga penalty kicks edited/1xx/alvarez.mp4: 384x640 1 ball, 4 goalframes, 1 goalie, 1 kicker

In [ ]:
import os
import shutil

# Root folder where your original data images/texts are
SOURCE_ROOT = "/runs/baseline/laliga penalty kicks edited"

# Folder where YOLO saved all predicted .txt files (flat)
PRED_LABELS = "runs/predict/labels"

# The subfolders (classes) you have
SUBFOLDERS = ["1xx", "x1x", "xx1"]

# Extensions of your images (adjust if you use png, jpeg, etc.)
IMAGE_EXTENSIONS = [".jpg", ".jpeg", ".png"]

for txt_file in os.listdir(PRED_LABELS):
    if not txt_file.endswith(".txt"):
        continue

    # Get base filename without extension
    base_name = os.path.splitext(txt_file)[0]

    found_folder = None

    # Search in each subfolder for matching image file
    for folder in SUBFOLDERS:
        folder_path = os.path.join(SOURCE_ROOT, folder)

        for ext in IMAGE_EXTENSIONS:
            image_path = os.path.join(folder_path, base_name + ext)
            if os.path.exists(image_path):
                found_folder = folder
                break
        if found_folder:
            break

    if found_folder is None:
        print(f"WARNING: Could not find matching image for {txt_file}. Skipping.")
        continue

    # Create target subfolder under PRED_LABELS if not exist
    target_dir = os.path.join(PRED_LABELS, found_folder)
    os.makedirs(target_dir, exist_ok=True)

    # Move the txt file
    src_path = os.path.join(PRED_LABELS, txt_file)
    dst_path = os.path.join(target_dir, txt_file)
    shutil.move(src_path, dst_path)

    print(f"Moved {txt_file} to {target_dir}")


FileNotFoundError: [Errno 2] No such file or directory: 'runs/predict/labels'

In [27]:
import numpy as np
import os

def parse_frame_file(filepath):
    """
    Parse one frame's .txt file into a 16D feature vector.
    """
    class_boxes = {0: [], 1: [], 2: [], 3: []}

    with open(filepath) as f:
        for line in f:
            parts = line.strip().split()
            if not parts:
                continue
            class_id = int(parts[0])
            features = list(map(float, parts[1:]))
            class_boxes[class_id].append(features)

    frame_features = []

    # Ball (0): use first or zeros
    if class_boxes[0]:
        frame_features.extend(class_boxes[0][0])
    else:
        frame_features.extend([0.0, 0.0, 0.0, 0.0])

    # Goalframe (1): average all detections
    if class_boxes[1]:
        avg_goalframe = np.mean(class_boxes[1], axis=0)
        frame_features.extend(avg_goalframe)
    else:
        frame_features.extend([0.0, 0.0, 0.0, 0.0])

    # Goalie (2): use first or zeros
    if class_boxes[2]:
        frame_features.extend(class_boxes[2][0])
    else:
        frame_features.extend([0.0, 0.0, 0.0, 0.0])

    # Kicker (3): use first or zeros
    if class_boxes[3]:
        frame_features.extend(class_boxes[3][0])
    else:
        frame_features.extend([0.0, 0.0, 0.0, 0.0])

    return frame_features


In [33]:
import os
root_dir = "runs/predict"
for path, dirs, files in os.walk(root_dir):
    print(f"\nPath: {path}")
    print(f"Dirs: {dirs}")
    print(f"Files: {files}")


Path: runs/predict
Dirs: ['labels_xx1', 'labels_1xx', 'labels_x1x']
Files: []

Path: runs/predict/labels_xx1
Dirs: ['labels']
Files: ['lewan.avi', 'celso (2).avi', 'griezmann.avi', 'olmo.avi', 'isco.avi', 'vini.avi', 'alvarez.avi', 'mbappe (2).avi', 'sorloth.avi', 'mbappe.avi', 'celso.avi', 'mbappe (3).avi', 'isco (2).avi']

Path: runs/predict/labels_xx1/labels
Dirs: []
Files: ['mbappe (3)_22.txt', 'isco_38.txt', 'lewan_39.txt', 'isco_65.txt', 'mbappe (2)_27.txt', 'lewan_34.txt', 'mbappe (2)_39.txt', 'celso (2)_6.txt', 'isco_52.txt', 'mbappe (2)_43.txt', 'celso (2)_38.txt', 'mbappe_16.txt', 'isco_68.txt', 'lewan_44.txt', 'sorloth_29.txt', 'celso_3.txt', 'lewan_62.txt', 'mbappe (3)_26.txt', 'mbappe_38.txt', 'mbappe (3)_33.txt', 'celso (2)_31.txt', 'vini_23.txt', 'isco (2)_5.txt', 'lewan_32.txt', 'isco (2)_7.txt', 'alvarez_4.txt', 'olmo_19.txt', 'celso_5.txt', 'isco_14.txt', 'celso (2)_61.txt', 'lewan_111.txt', 'sorloth_39.txt', 'isco (2)_17.txt', 'mbappe_28.txt', 'griezmann_13.txt', 'c

In [34]:
X = []
y = []

# Root data folder
root_dir = "runs/predict"

for label_folder in os.listdir(root_dir):
    label_path = os.path.join(root_dir, label_folder)
    if not os.path.isdir(label_path):
        continue

    # Determine label from folder name
    if "1xx" in label_folder:
        label = 0
    elif "x1x" in label_folder:
        label = 1
    elif "xx1" in label_folder:
        label = 2
    else:
        continue

    # Go into the inner 'labels' folder
    inner_labels_path = os.path.join(label_path, "labels")
    if not os.path.isdir(inner_labels_path):
        print(f"WARNING: No labels folder in {label_path}")
        continue

    for filename in os.listdir(inner_labels_path):
        if not filename.endswith(".txt"):
            continue

        filepath = os.path.join(inner_labels_path, filename)
        features = parse_frame_file(filepath)
        X.append(features)
        y.append(label)

X = np.array(X)
y = np.array(y)

print("Data shape:", X.shape)
print("Labels shape:", y.shape)

Data shape: (1204, 16)
Labels shape: (1204,)


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Pipeline
clf = make_pipeline(
    StandardScaler(),
    SVC(kernel='rbf', C=1.0, gamma='scale')
)
clf.fit(X_train, y_train)

# Evaluate
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      0.96      0.98        80
           1       0.97      0.92      0.95        39
           2       0.96      1.00      0.98       122

    accuracy                           0.98       241
   macro avg       0.98      0.96      0.97       241
weighted avg       0.98      0.98      0.97       241

